# Lightgbm

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import lightgbm

from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold
from IPython.display import display

In [2]:
train = joblib.load('models/train.joblib')
targets = train['TARGET']

train_ids = train['SK_ID_CURR']
train = train.drop(columns=['SK_ID_CURR', 'TARGET'])



In [4]:
test = joblib.load('models/test.joblib')
test_ids = test['SK_ID_CURR']
test = test.drop(columns=['SK_ID_CURR'])

In [5]:
# Convert to np arrays
features = np.array(train)
test_features = np.array(test)

In [8]:
parameters = {
    'task': 'train',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'nthread': 6,
    'max_depth': 10
    #'device': 'gpu'
}

k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

valid_scores = []
train_scores = []

for train_indices, valid_indices in k_fold.split(features, targets):
    # Training data for the fold
    train_features, train_labels = features[train_indices], targets[train_indices]
    # Validation data for the fold
    valid_features, valid_labels = features[valid_indices], targets[valid_indices]
    
    d_train = lightgbm.Dataset(train_features, label=train_labels)
    d_valid = lightgbm.Dataset(valid_features, label=valid_labels)
    
    model = lightgbm.train(parameters,
                       d_train,
                       verbose_eval=100,
                       valid_sets=[d_valid],
                       num_boost_round=20000,
                       early_stopping_rounds=200)

    #alid_scores.append(model.best_score_['valid']['auc'])
    #rain_scores.append(model.best_score_['train']['auc'])
    
    #_pred = model.predict(test)
    

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.747797
[200]	valid_0's auc: 0.755032
[300]	valid_0's auc: 0.755253
[400]	valid_0's auc: 0.75535
Early stopping, best iteration is:
[264]	valid_0's auc: 0.7556
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.750617
[200]	valid_0's auc: 0.758194
[300]	valid_0's auc: 0.759359
[400]	valid_0's auc: 0.759303
[500]	valid_0's auc: 0.759168
Early stopping, best iteration is:
[313]	valid_0's auc: 0.759424
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.751074
[200]	valid_0's auc: 0.75871
[300]	valid_0's auc: 0.759203
[400]	valid_0's auc: 0.758847
Early stopping, best iteration is:
[289]	valid_0's auc: 0.759327
Training until validation scores don't improve for 200 rounds.
[100]	valid_0's auc: 0.748821
[200]	valid_0's auc: 0.756145
[300]	valid_0's auc: 0.756399
[400]	valid_0's auc: 0.756687
[500]	valid_0's auc: 0.756371
[600]	valid_0's a